<a href="https://colab.research.google.com/github/joyce-ol/Predicting_house_price/blob/main/NYC_Airbn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inspiration
What can we learn about different hosts and areas?
What can we learn from predictions? (ex: locations, prices, reviews, etc)
Which hosts are the busiest and why?
Is there any noticeable difference of traffic among different areas and what could be the reason for it?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('AB_NYC_2019.csv')

Since 2008, guests and hosts have used Airbnb to expand on traveling possibilities and present more unique, personalized way of experiencing the world. This dataset describes the listing activity and metrics in NYC, NY for 2019.

In [ ]:
df.head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  2018-10-19   
1  Entire home/apt    225               1                 45  2019-05-21   
2     Private room    150               3                  0         NaN   
3  Entire home/apt     89               1                270  2019-07-05   
4  Entire home/apt     80              10                  9  2018-11-19   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               0.21                               6               365  
1               0.38                               2               355  
2                NaN                               1               365  
3               4.64                               1               194  
4               0.10                               1                 0

In [ ]:
df.shape

(48895, 16)

In [ ]:
df.nunique()

id                                48895
name                              47905
host_id                           37457
host_name                         11452
neighbourhood_group                   5
neighbourhood                       221
latitude                          19048
longitude                         14718
room_type                             3
price                               674
minimum_nights                      109
number_of_reviews                   394
last_review                        1764
reviews_per_month                   937
calculated_host_listings_count       47
availability_365                    366
dtype: int64

In [ ]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [ ]:
df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [ ]:
df.describe()

id       host_id      latitude     longitude         price  \
count  4.889500e+04  4.889500e+04  48895.000000  48895.000000  48895.000000   
mean   1.901714e+07  6.762001e+07     40.728949    -73.952170    152.720687   
std    1.098311e+07  7.861097e+07      0.054530      0.046157    240.154170   
min    2.539000e+03  2.438000e+03     40.499790    -74.244420      0.000000   
25%    9.471945e+06  7.822033e+06     40.690100    -73.983070     69.000000   
50%    1.967728e+07  3.079382e+07     40.723070    -73.955680    106.000000   
75%    2.915218e+07  1.074344e+08     40.763115    -73.936275    175.000000   
max    3.648724e+07  2.743213e+08     40.913060    -73.712990  10000.000000   

       minimum_nights  number_of_reviews  reviews_per_month  \
count    48895.000000       48895.000000       38843.000000   
mean         7.029962          23.274466           1.373221   
std         20.510550          44.550582           1.680442   
min          1.000000           0.000000           0.010000   
25%          1.000000           1.000000           0.190000   
50%          3.000000           5.000000           0.720000   
75%          5.000000          24.000000           2.020000   
max       1250.000000         629.000000          58.500000   

       calculated_host_listings_count  availability_365  
count                    48895.000000      48895.000000  
mean                         7.143982        112.781327  
std                         32.952519        131.622289  
min                          1.000000          0.000000  
25%                          1.000000          0.000000  
50%                          1.000000         45.000000  
75%                          2.000000        227.000000  
max                        327.000000        365.000000

In [ ]:
# Strip whitespace and convert string columns to lowercase
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip().str.lower()


In [ ]:
# handling missing values
df = df.dropna(subset=['name','host_name'])

#name and host_name are esstenial identifer so it is safer to drop them

In [ ]:
# For 'reviews_per_month', fill with 0 when there are no reviews
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [ ]:
# Convert 'last_review' column to datetime format
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# Filling last_review with a sentinel date
df.fillna({'last_review': pd.Timestamp('1900-01-01')}, inplace=True)

In [ ]:
print("\nUpdated data types:")
df.dtypes


Updated data types:


id                                         int64
name                                      object
host_id                                    int64
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

In [ ]:
df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [ ]:
# Check for duplicate rows based on all columns
duplicates = df.duplicated()
print(f"Total duplicate rows: {duplicates.sum()}")

Total duplicate rows: 0


In [ ]:
# clean text field
df['name'] = df['name'].str.strip()
df['host_name'] =df['host_name'].str.strip()
df['neighbourhood_group'] = df['neighbourhood_group'].str.strip()
df['neighbourhood'] = df['neighbourhood'].str.strip()
df['room_type'] = df['room_type'].str.strip()

In [ ]:
# Remove special characters from all object (string) columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.replace(r'[^\w\s]', '', regex=True)



In [ ]:
# handling outlier using IQR(interquartile range)
def cap_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return np.clip(series, lower_bound, upper_bound)



In [ ]:
df['price'] = cap_outliers(df['price'])
df['minimum_nights'] = cap_outliers(df['minimum_nights'])
df['availability_365'] = cap_outliers(df['availability_365'])


In [ ]:
df.describe()

id       host_id      latitude     longitude         price  \
count  4.889500e+04  4.889500e+04  48895.000000  48895.000000  48895.000000   
mean   1.901714e+07  6.762001e+07     40.728949    -73.952170    132.979753   
min    2.539000e+03  2.438000e+03     40.499790    -74.244420      0.000000   
25%    9.471945e+06  7.822033e+06     40.690100    -73.983070     69.000000   
50%    1.967728e+07  3.079382e+07     40.723070    -73.955680    106.000000   
75%    2.915218e+07  1.074344e+08     40.763115    -73.936275    175.000000   
max    3.648724e+07  2.743213e+08     40.913060    -73.712990    334.000000   
std    1.098311e+07  7.861097e+07      0.054530      0.046157     83.530504   

       minimum_nights  number_of_reviews                    last_review  \
count     48895.00000       48895.000000                          48895   
mean          3.82532          23.274466  1994-05-05 21:06:35.598732032   
min           1.00000           0.000000            1900-01-01 00:00:00   
25%           1.00000           1.000000            2016-03-24 00:00:00   
50%           3.00000           5.000000            2019-01-03 00:00:00   
75%           5.00000          24.000000            2019-06-19 00:00:00   
max          11.00000         629.000000            2019-07-08 00:00:00   
std           3.32072          44.550582                            NaN   

       reviews_per_month  calculated_host_listings_count  availability_365  
count       48895.000000                    48895.000000      48895.000000  
mean            1.090910                        7.143982        112.781327  
min             0.000000                        1.000000          0.000000  
25%             0.040000                        1.000000          0.000000  
50%             0.370000                        1.000000         45.000000  
75%             1.580000                        2.000000        227.000000  
max            58.500000                      327.000000        365.000000  
std             1.597283                       32.952519        131.622289

In [ ]:
df.to_csv("cleaned_AB_NYC_2019.csv", index=False)
